## **Project Title:** Analyzing Customer Sales Data for a Retail Store
**Authorized By:** Azeem Data Scientist\
**Date:** 11 / 27 / 2025\
**Email:** azeem.datascientist@gmail.com

#### **Business Problem:** A local retailer, "Islamabad Mart," has given you a messy sales dataset. They want to understand their business performance to make better decisions. They have three key questions:

1. Who are our top 5 customers by total sales value?

2. What is our total revenue, and which product category is the most profitable?

3. On which day of the week do we make the most sales?

##### **The Data (islamabad_mart_sales.csv):**

In [26]:
# data_generator.py - RUN THIS SCRIPT TO CREATE YOUR DATASET
import pandas as pd
import numpy as np

np.random.seed(42)  # For reproducible results
num_records = 1000

data = {
    'OrderID': range(1000, 1000 + num_records),
    'Customer Name': np.random.choice(['Ali Raza', 'Fatima Khan', 'Ahmed Siddiqui', 'Ayesha Malik', 'Bilal Yousuf', 'Zainab Arif', 'Usman Chaudhry', 'Sana Farooqi'], num_records),
    'Product Category': np.random.choice(['Electronics', 'Clothing', 'Groceries', 'Home & Kitchen'], num_records),
    'Sales Amount (PKR)': np.round(np.random.uniform(500, 50000, num_records), 2),
    'Date': pd.date_range('2023-01-01', periods=num_records, freq='D')
}

df = pd.DataFrame(data)

# INTRODUCE REAL-WORLD MESSINESS
# 1. Duplicate some rows
duplicates = df.sample(50)
df = pd.concat([df, duplicates], ignore_index=True)

# 2. Create inconsistent capitalization in names
df['Customer Name'] = df['Customer Name'].str.upper()

# 3. Add some missing values
df.loc[df.sample(30).index, 'Product Category'] = None

# Save to CSV
df.to_csv('islamabad_mart_sales.csv', index=False)
print("Dataset 'islamabad_mart_sales.csv' created! Your mission begins.")

Dataset 'islamabad_mart_sales.csv' created! Your mission begins.


##### 1. Data Loading and Inspection:

In [27]:
# load the dataset
df = pd.read_csv('islamabad_mart_sales.csv')
df.head()

,OrderID,Customer Name,Product Category,Sales Amount (PKR),Date
0,1000,USMAN CHAUDHRY,Clothing,9664.08,2023-01-01
1,1001,AYESHA MALIK,Groceries,27324.10,2023-01-02
2,1002,BILAL YOUSUF,Electronics,43710.82,2023-01-03
3,1003,USMAN CHAUDHRY,Electronics,36745.13,2023-01-04
4,1004,AHMED SIDDIQUI,Electronics,40424.78,2023-01-05


In [28]:
# whole information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   OrderID             1050 non-null   int64  
 1   Customer Name       1050 non-null   object 
 2   Product Category    1020 non-null   object 
 3   Sales Amount (PKR)  1050 non-null   float64
 4   Date                1050 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 41.1+ KB


In [29]:
# the dates are in string, so first we have to convert the string dates into actual dates
df['Date'] = pd.to_datetime(df['Date'])

##### 2. Data Cleaning (The Python Function):

In [30]:
def clean_customer_names(series):

    """
    This function can take a single column as a argument then the function can convert each string
    value to title case
    """

    result = series.str.title()
    return result

# apply that function on a customer name column
df['Customer Name'] = clean_customer_names(df['Customer Name'])

In [31]:
# verify the results
df['Customer Name'].head(3)

0    Usman Chaudhry
1      Ayesha Malik
2      Bilal Yousuf
Name: Customer Name, dtype: object

In [32]:
# handle missing values;
# fill all the missing values with the unknown string
df['Product Category'].fillna('Unknown', inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_11748\2200469304.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Product Category'].fillna('Unknown', inplace=True)


In [33]:
# remove the duplicate records from the dataset to ensure data validity
df = df.drop_duplicates()

In [34]:
# dimension of the dataset
df.shape

(1002, 5)

##### 3. Business Analysis (The Core Pandas):

Q1: Top 5 Customers:

In [35]:
# grouped by Customer Name to find the top 5 customers
# sort_values(ascending = False) -> can sort the values to highest first
df.groupby(['Customer Name'])['Sales Amount (PKR)'].sum().sort_values(ascending=False).head(5)

Customer Name
Ayesha Malik    3792625.49
Sana Farooqi    3419662.05
Ali Raza        3408271.56
Bilal Yousuf    3262560.91
Fatima Khan     3176709.50
Name: Sales Amount (PKR), dtype: float64

Q2: Revenue & Top Category:

In [36]:
# calculate the total revenue of the business
total_revenue = df['Sales Amount (PKR)'].sum()
print(f'The Total Revenue is',total_revenue,'PKR')

The Total Revenue is 25682709.52 PKR


In [37]:
# product category with the highest total sales
df.groupby('Product Category')['Sales Amount (PKR)'].sum().sort_values(ascending=False).head(1)

Product Category
Electronics    6778531.57
Name: Sales Amount (PKR), dtype: float64

Q3: Sales by Day of Week

In [38]:
# feature engineering;
# convert the dates into days in a new column -> `DayOfWeek`
df['DayOfWeek'] = df['Date'].dt.day_name()

In [39]:
# find the day with the highest total sales
daily_sales = df.groupby('DayOfWeek')['Sales Amount (PKR)'].sum()

In [40]:
# find the day with the maximum sales
day_max_sales = daily_sales.idxmax()
max_sales_amount = daily_sales.max()

# results
print(f'Daily Total Sales:\n{daily_sales}')
print(f'\nThe day with the most sales is {day_max_sales} with total sales of PKR {max_sales_amount}')

Daily Total Sales:
DayOfWeek
Friday       3702007.48
Monday       3576420.91
Saturday     3870226.07
Sunday       3608988.98
Thursday     3681509.14
Tuesday      3545612.74
Wednesday    3697944.20
Name: Sales Amount (PKR), dtype: float64

The day with the most sales is Saturday with total sales of PKR 3870226.07


In [41]:
weekly_revenue_percentage = (daily_sales / total_revenue) * 100
print(f'Weekly Revenue Contribution (Percentage):\n{weekly_revenue_percentage}')

Weekly Revenue Contribution (Percentage):
DayOfWeek
Friday       14.414396
Monday       13.925403
Saturday     15.069384
Sunday       14.052213
Thursday     14.334582
Tuesday      13.805447
Wednesday    14.398575
Name: Sales Amount (PKR), dtype: float64
